In [1]:
import sys
import os
import yaml
import getpass

#adding prognos tools
sys.path.insert(0, "/home/jose-luis/Envs/niva_prognos/PROGNOS/")
from prognos_tools.encrypt import decryptString,encryptString
from prognos_tools.Basin import Basin

from geonorge_scraping import Download

geonorge_encrypted=b'gAAAAABc2SuJN0rMAgv8fEeN9U_YTle-kcHc1Hy63ON7Y_LpkKAfmWBY01Eud7yA_CvU_rsoM_VstFzTLsX28BGb7d9aVIfqnp76smF4rn0sF8Sp4fRZnHI-B8TfVlew-qcDmewcvCj8'
key = getpass.getpass('Password: ')
geonorge_credentials = yaml.safe_load(decryptString(geonorge_encrypted,key))

# The encryption key has a 90 days timespan and might need to be re-encrypted
# with open('geocredentials.yaml','w') as f:
#     f.write(yaml.dump(geonorge_credentials))
    
# Basin.callPopen('gcloud kms encrypt --location global --keyring jlg-keyring --key prognos-key --plaintext-file geocredentials.yaml --ciphertext-file geocredentials.yaml.encrypted')

Password:  ·······


# Setting up vm with google chrome installed to crawl through geonorge.no

In [2]:
machineInfo={}
#Vm characteristics
machineInfo['instance'] = 'geonorgecito'
machineInfo['username'] = 'jose-luis'
machineInfo['image'] = 'debian-9-stretch-v20190423'
machineInfo['imageProject'] = 'debian-cloud'
machineInfo['keyDir'] = '/home/jose-luis/.ssh/geonorge_crawler'
machineInfo['region'] = 'europe-west2-a'
machineInfo['machineType'] = 'n1-standard-4'
machineInfo['keyFile'] = F'{os.path.join(machineInfo["keyDir"],machineInfo["username"])}'
machineInfo['pubKeyFile'] = F'{machineInfo["keyFile"] + ".pub"}'

geonorge = Basin(machineInfo)

geonorge.vmScript = '''#!/bin/bash
sudo apt-get update
yes | sudo apt-get install gdal-bin libgdal-dev python-gdal tmux unzip
yes | sudo apt-get install libxss1 libappindicator1 libindicator7
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
yes | sudo apt-get install fonts-liberation libappindicator3-1 libasound2 libatk-bridge2.0-0 libgtk-3-0 libx11-xcb1 libxtst6 xdg-utils
yes | sudo dpkg -i google-chrome*.deb
wget https://chromedriver.storage.googleapis.com/2.42/chromedriver_linux64.zip && unzip chromedriver_linux64.zip
yes | sudo apt-get install python-pip parallel python3-pip
wget https://github.com/espena/sosicon/blob/master/bin/cmd/linux64/sosicon?raw=true -O sosicon && chmod +x sosicon

pip3 install selenium feedparser pyyaml

export GCSFUSE_REPO=gcsfuse-`lsb_release -c -s`
echo "deb http://packages.cloud.google.com/apt $GCSFUSE_REPO main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

yes | sudo apt-get update
yes | sudo apt-get install gcsfuse

rm -rf dummy
mkdir dummy
gcsfuse jlg-bucket dummy
rm -f ./dummy/sosicon
cp ./sosicon ./dummy
fusermount -u dummy
rm -rf dummy
'''

Basin.initString += '''--scopes=https://www.googleapis.com/auth/devstorage.read_write,\
https://www.googleapis.com/auth/cloudkms\n'''

geonorge.instantiate(fabfile='fabfile.py')
print("The ip of the google cloud instance is {}".format(geonorge.ip))

#Testing connection to instance
geonorge.callPopen('gcloud compute ssh {username}@{instance} --zone {region}'.format(
**geonorge.machineInfo))

#Installing dependencies necessary to crawl through geonorge
scriptsFolder='./scripts'
geonorge.runScript(saveFolder=scriptsFolder,scriptName='setup.sh')

KeyError: 'diskSize'

## Passing geonorge credentials and downloading land cover data

In [ ]:
#Putting the necessary files on the vm
Basin.callPopen('fab downloadFromGeonorge')

#Downloading data using geonorge_scraping.py utilities
geonorge.vmScript='''#!/usr/bin/python3
import geonorge_scraping as gs
google = gs.Download()
google.getUrls()
google.download(saveFolder='./data_AR5')
'''

geonorge.runScript(saveFolder=scriptsFolder,scriptName='download.py')

#Putting all downloads in a cloud bucket
geonorge.vmScript='''#!/bin/bash
tar -cf AR5.tar ./data_AR5
rm -rf upload
mkdir upload
gcsfuse jlg-bucket upload
rm -f ./upload/AR5.tar
cp AR5.tar ./upload/AR5.tar
fusermount -u ./upload'''

geonorge.runScript(saveFolder=scriptsFolder,scriptName='toCloudBucket.sh')

In [ ]:
geonorge.kill()